# Are the LLMs playing like a human? 
Goal: check if the LLMs have a behaviour similar to humans on Wikispeedia.

Strategy: make a LLM play to 2007 Wikispeedia and compare its answers with human paths.

Steps for a MWE:
1. decide which path to use (take one played a lot by people to have more data to compare with)
2. take a LLM
3. define the prompt
4. make it play
5. compare with paths of humans

Points to adjust:
- try different paths
- try different LLMs
- try different prompts


In [1]:
import ollama
from ollama import Client

# Create a client
client = Client(host='http://localhost:11434')

# Paths to use to try the LLMs
Let's find the most finished paths. 

In [2]:
import pandas as pd 
import os

DATA_PATH = 'data/wikispeedia_paths-and-graph/'

# load tsv files into pandas dataframes
path_finished = pd.read_csv(os.path.join(DATA_PATH, 'paths_finished.tsv'), sep='\t', comment='#', names=['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating'])
df_links = pd.read_csv(os.path.join(DATA_PATH, 'links.tsv'), sep='\t', comment='#', names=['linkSource', 'linkTarget'])
path_finished.path = path_finished.path.str.split(';')
path_finished['start'] = path_finished.path.str[0]
path_finished['end'] = path_finished.path.str[-1]

by_path = path_finished.groupby(by=['start', 'end']).apply(lambda x: x)
by_path

/tmp/ipykernel_25995/3733474754.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  by_path = path_finished.groupby(by=['start', 'end']).apply(lambda x: x)


hashedIpAddress  \
start                          end                                    
%E2%82%AC2_commemorative_coins Irish_Sea    32997  651ff0fa4fac4471   
10th_century                   11th_century 29758  516b61133d358ce1   
                                            29759  6b039e9953cf075e   
                                            29760  0aecf97906bcb41a   
                               Banknote     29761  32652d6d1c5d9351   
...                                                             ...   
Zulu                           Doom         50811  267f588369bcec64   
                               Jesus        50807  027433ec4cc9ff72   
                               Language     50806  4ee1908c061cfc53   
                               Slovenia     50812  401e0e507140865e   
                                            50813  767f706b73fa806a   

                                                    timestamp  durationInSec  \
start                          end                                             
%E2%82%AC2_commemorative_coins Irish_Sea    32997  1227628729             15   
10th_century                   11th_century 29758  1224623308              6   
                                            29759  1241187124              3   
                                            29760  1373439892              4   
                               Banknote     29761  1260397548             48   
...                                                       ...            ...   
Zulu                           Doom         50811  1248704813            455   
                               Jesus        50807  1368505587              7   
                               Language     50806  1260241074             29   
                               Slovenia     50812  1249233736             56   
                                            50813  1265042003             63   

                                                                                                path  \
start                          end                                                                     
%E2%82%AC2_commemorative_coins Irish_Sea    32997  [%E2%82%AC2_commemorative_coins, Ireland, Iris...   
10th_century                   11th_century 29758                       [10th_century, 11th_century]   
                                            29759                       [10th_century, 11th_century]   
                                            29760                       [10th_century, 11th_century]   
                               Banknote     29761  [10th_century, Maya_civilization, Silver, Coin...   
...                                                                                              ...   
Zulu                           Doom         50811  [Zulu, AK-47, Canada, <, <, English_language, ...   
                               Jesus        50807                        [Zulu, Christianity, Jesus]   
                               Language     50806                 [Zulu, English_language, Language]   
                               Slovenia     50812  [Zulu, South_Africa, Continent, Europe, Slovenia]   
                                            50813  [Zulu, South_Africa, Mediterranean_Sea, Slovenia]   

                                                   rating  \
start                          end                          
%E2%82%AC2_commemorative_coins Irish_Sea    32997     1.0   
10th_century                   11th_century 29758     1.0   
                                            29759     5.0   
                                            29760     1.0   
                               Banknote     29761     3.0   
...                                                   ...   
Zulu                           Doom         50811     3.0   
                               Jesus        50807     NaN   
                               Language     50806     NaN   
                               Slovenia     50812     1.0   
  

In [3]:

by_path_counts = path_finished.groupby(by=['start', 'end']).size().sort_values(ascending=False)
by_path_counts[:50]

start                     end              
Asteroid                  Viking               1043
Brain                     Telephone            1040
Theatre                   Zebra                 905
Pyramid                   Bean                  642
Batman                    Wood                  148
Bird                      Great_white_shark     138
Batman                    The_Holocaust         119
Bird                      Adolf_Hitler          107
Beer                      Sun                    99
Batman                    Banana                 69
Cat                       Computer               57
                          Microsoft              56
Dog                       Telephone              53
Flower                    Adolf_Hitler           51
Automobile                Pluto                  47
Dog                       Venus                  47
Batman                    Bible                  43
Aluminium_chloride        Parrot                 42
England             

In [4]:
def get_links(article):
    """
    Get all links from a given article

    Args:
    article (str): article name

    Returns:
    list: list of articles linked to
    """
    return df_links[df_links.linkSource == article].linkTarget.values.tolist()

In [5]:
def llm_input(target, links):
    """
    Format input for LLM

    Args:
    target (str): target article
    links (list): list of articles linked to

    Returns:
    str: formatted input
    """

    return f"""

Target word: [{target}]

Available options: [{links}]

RESPECT THIS FORMAT WHEN ANSWERING:

Reasoning: [REASONING]

Answer: Hence the choice is: '[ANSWER]' """
    
llm_input('George_Washington', ['Able_Archer_83', 'Afghanistan', 'Estonia', 'Europe', 'Finland', 'France', 'French_language', 'George_W._Bush', 'Hungary', 'September_11,_2001_attacks', 'United_States'])


"\n\nTarget word: [George_Washington]\n\nAvailable options: [['Able_Archer_83', 'Afghanistan', 'Estonia', 'Europe', 'Finland', 'France', 'French_language', 'George_W._Bush', 'Hungary', 'September_11,_2001_attacks', 'United_States']]\n\nRESPECT THIS FORMAT WHEN ANSWERING:\n\nReasoning: [REASONING]\n\nAnswer: Hence the choice is: '[ANSWER]' "

In [6]:
get_links('George_Washington')

['Abraham_Lincoln',
 'Alexander_Hamilton',
 'American_Revolutionary_War',
 'Church_of_England',
 'France',
 'Great_Lakes',
 'Hippopotamus',
 'John_Adams',
 'Lead',
 'Mount_Rushmore',
 'NATO',
 'New_York_City',
 'Pneumonia',
 'President_of_the_United_States',
 'Rhode_Island',
 'Supreme_Court_of_the_United_States',
 'Theodore_Roosevelt',
 'Thomas_Jefferson',
 'United_States',
 'United_States_Constitution',
 'West_Virginia']

In [7]:
response = client.chat(model='mistral', messages=[
            {
                'role': 'user',
                'content': """We now play the following game:

I will give you a target word and a list from which you can choose an option. If the available options contains the target word, you choose it. Otherwise you choose the option that is most similar to it. Before starting, I give you one examples, then it's your turn:

you need to follow the same format as the example below:
Target word: George_Washington

Available options: [Able_Archer_83, Afghanistan, , Estonia, Europe, Finland, France, French_language, George_W._Bush, Hungary, September_11,_2001_attacks, United_States]

Reasoning: I need to find something inside the list related to the target: 'George_Washington'. George Washington was the first president of United States and he lived in United States.

Answer: Hence the answer is: 'United_States'.
""",
            },
        ])

print(response['message']['content'])

 Target word: Mozart

   Available options: [Albert_Einstein, Beethoven, Brahms, Chopin, Debussy, Haydn, Liszt, Mendelssohn, Mozartkugel, Rachmaninoff, Schubert, Vienna]

   Answer: 'Mozartkugel' (This is a confectionery made in Salzburg, Austria, named after Mozart. Although not a person or location directly associated with Mozart, it is related to him through a sweet creation.)


In [8]:
def llm_player(start, end):
    """
    Play the game of Wikispeedia

    Args:
    start (str): start article
    end (str): end article

    Returns:
    list: path taken
    """
    path = [start]
    current = start
    while current != end:
        links = get_links(current)
        response = client.chat(model='mistral', messages=[
            {
                'role': 'user',
                'content': llm_input(end, links),
            },
        ])
        # print(response['message']['content'])
        # get the last paragraph
        answer = response['message']['content'].split('\n')[-1]
        for link in links:
            if link in answer:
                current = link
                print(current)
                path.append(current)
    return path

llm_player('Asteroid', 'Viking')

Venus
European_Space_Agency
Denmark
Finland
Norway
Sweden
Viking


['Asteroid',
 'Venus',
 'European_Space_Agency',
 'Denmark',
 'Finland',
 'Norway',
 'Sweden',
 'Viking']

# Ideas of prompt:

- from AI vs human project:
`We now play the following game:

I will give you a target word and a list from which you can choose an option. If the list contains the target word, you choose it. Otherwise you choose the option that is most similar to it. Before starting, I give you one examples, then it's your turn:

EXAMPLE:
Target word: George_Washington

Available options: [[Able_Archer_83, Afghanistan, , Estonia, Europe, Finland, France, French_language, George_W._Bush, Hungary, September_11,_2001_attacks, United_States]]

Reasoning: I need to find something inside the list related to the target: 'George_Washington'. George Washington was the first president of United States and he lived in United States.

Answer: Hence the answer is: 'United_States'.

YOUR TURN:

Target word: {TARGET}

Available options: [[{LIST OF LINKS}]]

Reasoning: [REASONING]

Answer: Hence the choice is: '[ANSWER]'`


- ordering the options by order of appearence in text, to reproduce the fact that this is what humans see first.
`We now play the following game:

I will give you a target word and a list (in no specific order) from which you can choose an option. If the list contains the target word, you choose it. Otherwise you choose the option that is most similar to it. Before starting, I give you one examples, then it's your turn:

EXAMPLE:
Target word: George_Washington

Available options: [change]

Reasoning: I need to find something inside the list related to the target: 'George_Washington'. George Washington was the first president of United States and he lived in United States.

Answer: Hence the answer is: 'United_States'.

YOUR TURN:

Target word: {TARGET}

Available options: [[{LIST OF LINKS}]]

Reasoning: [REASONING]

Answer: Hence the choice is: '[ANSWER]'`



- explicit further the game 
`We now play the following game: I give you a target encyclopedia article, the current article you are in and the list of other article titles present in this current article. The final goal is to reach the target article as fast as possible by navigating from article to article. Your task right now is to choose an article from the list I give you that brings you closer to the target. If the target is present in the list, return the target.
Before starting, I give you one examples, then it's your turn:

EXAMPLE:
Target word: George_Washington

Available options: [change]

Reasoning: I need to find something inside the list related to the target: 'George_Washington'. George Washington was the first president of United States and he lived in United States.

Answer: Hence the answer is: 'United_States'.

YOUR TURN:

Target word: {TARGET}

Available options: [[{LIST OF LINKS}]]

Reasoning: [REASONING]

Answer: Hence the choice is: '[ANSWER]'`